In [35]:
import csv
import cv2
import numpy as np
import librosa
import librosa.display
import wave
import os
import matplotlib.pyplot as plt
import multiprocessing 
import pickle

In [36]:
SOURCE_FOLDER_TRAIN= '../AudioData/Cats-Vs-Dogs/'
OUTPUT_FOLDER_TRAIN = '../GeneratedData/Cats-Vs-Dogs/'

In [37]:
FileName =input("Type Filename befor the '.' => ")
SpectrumVariables = pickle.load(open( FileName+'.SpecVar', "rb" ) )
pickle.dump(SpectrumVariables, open(os.path.join(OUTPUT_FOLDER_TRAIN,'Main.SpecVar'), "wb" ))
SpectrumVariables

Type Filename befor the '.' => Standard


{'RESOLUTION': 224,
 'SAMPLE_RATE': 48000,
 'N_FFT': 1536,
 'HOP_LENGTH': 512,
 'FMIN': 10,
 'FMAX': 21000,
 'N_MELS': 224,
 'POWER': 2.0}

In [38]:
def log_mel_spec_tfm(src_path, dst_path):
    #print(src_path)
    fname=src_path.split('/')[-1];
    x=[]
    sample_rate=0
    try:
        channels ,sample_rate_in = librosa.load(src_path,mono=False)
    except:
        print(fname+" Could not be computed!")
        return
    if(channels.ndim==1):
        channels= channels.reshape((1,(len(channels))))
    channelcounter=0
    for x in channels:
        channelcounter+=1
        
        RESOLUTION=SpectrumVariables["RESOLUTION"] 
        SAMPLE_RATE=SpectrumVariables["SAMPLE_RATE"]
        N_FFT=SpectrumVariables["N_FFT"]
        HOP_LENGTH= SpectrumVariables["HOP_LENGTH"]
        FMIN=SpectrumVariables["FMIN"]
        FMAX=SpectrumVariables["FMAX"]
        N_MELS=SpectrumVariables["N_MELS"]
        POWER=SpectrumVariables["POWER"]
        audio_data = librosa.resample(x, sample_rate_in, SAMPLE_RATE)
        mel_spec_power = librosa.feature.melspectrogram(audio_data, sr=SAMPLE_RATE, n_fft=N_FFT, 
                                                        hop_length=HOP_LENGTH, 
                                                        n_mels=N_MELS, power=POWER,
                                                       fmin=FMIN,fmax=FMAX)
        mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
        image = mel_spec_db; # convert to float
        image -= image.min() # ensure the minimal value is 0.0
        image /= image.max() # maximum value in image is now 1.0
        image*=256;
        img = image.astype(np.uint8)
        colorPic = cv2.applyColorMap(img, cv2.COLORMAP_BONE)
        #cv2.imshow('dst_rt', colorPic)
        #cv2.waitKey(0)
        count=0
        for i in range(int(np.floor(colorPic.shape[1]/RESOLUTION))):
            count+=1
            startx=RESOLUTION*i
            stopx=RESOLUTION*(i+1)
            OutputImage = cv2.resize(colorPic[:,startx:stopx,:],(RESOLUTION,RESOLUTION))
            plt.imsave(os.path.join(dst_path,(fname.replace(".wav",'-').replace(".m4a",'-').replace(".mp3",'-') +str(i)+'-ch-'+str(channelcounter)+ '.png')), OutputImage) 
        if(count==0):
            print(src_path)
            #print("WARNING: => Jumped A file. If you see this often, increas sampleRate or Hop length.")

In [39]:
def ComputeSpectrograms(IN_FOLDER,OUT_FOLDER):
    print("I will print the file path to files that are too short to fit into one full picture.")
    SourceFoldersLabels = [f.path for f in os.scandir(IN_FOLDER) if f.is_dir()]
    for path in SourceFoldersLabels:
        FileList = [f.path for f in os.scandir(path) if f.is_file() and (f.name.endswith(".wav") or f.name.endswith(".m4a") or f.name.endswith(".mp3"))]
        Label = path.split('/')[-1]
        outFolder = os.path.join(OUT_FOLDER,Label)
        if not os.path.exists(outFolder):
            os.makedirs(outFolder)
        with multiprocessing.Pool(12) as p:
            p.starmap(log_mel_spec_tfm, [(f,outFolder) for f in FileList])
        print("Finished this pool. Now next Folder!")
    print("Done generating spectra!")  

In [41]:
ComputeSpectrograms(SOURCE_FOLDER_TRAIN,OUTPUT_FOLDER_TRAIN)

../AudioData/Cats-Vs-Dogs/Dogs/231762__davidmenke__fx-dog-barking.wav
../AudioData/Cats-Vs-Dogs/Dogs/231762__davidmenke__fx-dog-barking.wav
../AudioData/Cats-Vs-Dogs/Dogs/163459__littlebigsounds__lbs-fx-dog-small-alert-bark001.wav
../AudioData/Cats-Vs-Dogs/Dogs/163459__littlebigsounds__lbs-fx-dog-small-alert-bark001.wav
Finished this pool. Now next Folder!
../AudioData/Cats-Vs-Dogs/Cats/64006__department64__kitten11.wav
../AudioData/Cats-Vs-Dogs/Cats/461545__slv1__cat-meowing.wav
../AudioData/Cats-Vs-Dogs/Cats/64006__department64__kitten11.wav
../AudioData/Cats-Vs-Dogs/Cats/461545__slv1__cat-meowing.wav
../AudioData/Cats-Vs-Dogs/Cats/362327__vlcikamen__xoraya-begging-for-fish.mp3
../AudioData/Cats-Vs-Dogs/Cats/362327__vlcikamen__xoraya-begging-for-fish.mp3
../AudioData/Cats-Vs-Dogs/Cats/415209__inspectorj__cat-screaming-a.wav
../AudioData/Cats-Vs-Dogs/Cats/362953__josepharaoh99__cat-call-meow.mp3
../AudioData/Cats-Vs-Dogs/Cats/274151__darcydunes__cat-meow-short.wav
../AudioData/Cats-Vs

In [42]:
print("Done")

Done
